In [17]:
import pandas as pd
import numpy as np
from datetime import timedelta
import zenhan
from jeraconv import jeraconv
j2w = jeraconv.J2W()

import subprocess
import os

import requests
from bs4 import BeautifulSoup
import re

In [18]:
resposne = requests.get("https://www.gyoukaku.go.jp/hotline/index.html")
soup = BeautifulSoup(resposne.text, 'html.parser')
pattern = re.compile("\.xlsx$")

URL_LIST = []
for link in soup.find_all('a'):
	if pattern.search(link.get('href')):
		URL_LIST.append(
			os.path.join(
				"./rawdata/",
				os.path.basename(link.get('href'))
			)
		)

FILE_LIST = []
ITEM_NUM = len(URL_LIST)//2

for i in range(0, ITEM_NUM):
	FILE_LIST.append(
		{
			"type": "規制改革" if i < ITEM_NUM //2 else "行政改革",
			"teian": URL_LIST[i],
			"kaitou": URL_LIST[i+ITEM_NUM],
			"year": int(re.search("R([0-9]+)\.", URL_LIST[i]).group(1)),
			"headoffset_teian": 3 if i < ITEM_NUM//2 else 1,
			"headoffset_kaitou":2
		}
	)
	if i >= ITEM_NUM//2 and FILE_LIST[-1]["year"] == 2:
		FILE_LIST[-1]["headoffset_teian"] = 2
print(FILE_LIST)

JSON_SUFFIX="_processed"

[{'type': '規制改革', 'teian': './rawdata/02_teian_kisei_20230829_R5.xlsx', 'kaitou': './rawdata/18_kaitou_kisei_20230829_R5.xlsx', 'year': 5, 'headoffset_teian': 3, 'headoffset_kaitou': 2}, {'type': '規制改革', 'teian': './rawdata/04_teian_kisei_20230829_R4.xlsx', 'kaitou': './rawdata/20_kaitou_kisei_20230829_R4.xlsx', 'year': 4, 'headoffset_teian': 3, 'headoffset_kaitou': 2}, {'type': '規制改革', 'teian': './rawdata/06_teian_kisei_20230829_R3.xlsx', 'kaitou': './rawdata/22_kaitou_kisei_20230829_R3.xlsx', 'year': 3, 'headoffset_teian': 3, 'headoffset_kaitou': 2}, {'type': '規制改革', 'teian': './rawdata/08_teian_kisei_20230829_R2.xlsx', 'kaitou': './rawdata/24_kaitou_kisei_20230829_R2.xlsx', 'year': 2, 'headoffset_teian': 3, 'headoffset_kaitou': 2}, {'type': '行政改革', 'teian': './rawdata/10_teian_gyoukaku_20230829_R5.xlsx', 'kaitou': './rawdata/26_kaitou_gyoukaku_20230829_R5.xlsx', 'year': 5, 'headoffset_teian': 1, 'headoffset_kaitou': 2}, {'type': '行政改革', 'teian': './rawdata/12_teian_gyoukaku_20230829

In [19]:
baseurl = "https://www.gyoukaku.go.jp/hotline/img/"
for filepath in FILE_LIST:
    if not os.path.exists(filepath['teian']):
        url = baseurl + os.path.basename(filepath['teian'])
        subprocess.call(['wget', '--directory-prefix', './rawdata', url], shell=True)

    if not os.path.exists(filepath['kaitou']):
        url = baseurl + os.path.basename(filepath['kaitou'])
        subprocess.call(['wget', '--directory-prefix', './rawdata', url], shell=True)

## Helper関数

In [20]:
import datetime

## Cases
# STRING: '令和\n3年'
# INT: EXCEL DATE
# DATETIME: datetime

def normalize(data):
    if isinstance(data, int) or isinstance(data, float):
        return pd.to_datetime('1900-01-01') + pd.to_timedelta(data, 'D')
    elif isinstance(data, datetime.datetime):
        return data
    elif isinstance(data, str):
        oneline = zenhan.z2h(data.replace('\n', ''))
        Y, MD = oneline.split('年', 1)
        if Y.find("】") != -1:
            Y = Y.split("】")[1]
        try:
            Y = str(j2w.convert(Y + '年')) + '年'
        except:
            Y = str(j2w.convert('令和' + Y + '年')) + '年'
        date = datetime.datetime.strptime(Y+MD.split("日")[0], '%Y年%m月%d')
        return date
    else:
        return data

def conv_date(df, column):
    tmp = df[column].apply(normalize)
    return tmp

## 本題

In [21]:
df_list = []
for item in FILE_LIST:
	print(item["teian"], item["headoffset_teian"])
	df = pd.read_excel(item["teian"], header=item["headoffset_teian"])
	df.rename(columns={
		'所管省庁への　　検討要請日': '所管省庁への検討要請日',
		'所管省庁への\n検討要請日': '所管省庁への検討要請日',
		'所管省庁\nへの検討\n要請日': '所管省庁への検討要請日',
		'所管省庁\nへの検討要請日': '所管省庁への検討要請日',
		'内閣府での\n回答取りまとめ\n日':'回答取りまとめ日',
		'内閣府での\n回答取りまとめ日':'回答取りまとめ日',
		'提案事項名\n（タイトル）':'提案事項',
		'提案主体名\n（会社名・団体名）':'提案主体',
		'制度の所管省庁': '所管省庁',
		'ワーキング・グループにおける\n処理方針': 'ワーキング・グループにおける処理方針'
	}, inplace=True)
	print(df.columns)
	df['所管省庁への検討要請日'] = conv_date(df, '所管省庁への検討要請日')
	df['回答取りまとめ日'] = conv_date(df, '回答取りまとめ日')

	print(item["kaitou"])
	df2 = pd.read_excel(item["kaitou"], header=[0, 1])
	df2.rename(columns={
		'所管省庁への\n検討要請日': '所管省庁への検討要請日',
		'所管省庁\nへの検討\n要請日': '所管省庁への検討要請日',
		'所管省庁\nへの検討要請日': '所管省庁への検討要請日',
		'内閣府での\n回答取りまとめ\n日':'回答取りまとめ日',
		'提案事項名\n（タイトル）':'提案事項名',
		'提案主体名\n（会社名・団体名）':'提案主体名',
		'ワーキング・グループにおける\n処理方針': 'ワーキング・グループにおける処理方針'
	}, inplace=True)
	df2.columns = ["_".join(x) if isinstance(x[1], str) and x[1][0:3] != "Unn" else x[0].replace('\n','') for x in df2.columns.ravel()]
	df2['所管省庁への検討要請日'] = conv_date(df2, '所管省庁への検討要請日')
	df2['回答取りまとめ日'] = conv_date(df2, '回答取りまとめ日')

	df3 = pd.merge(df, df2, how='left', on='番号')
	df3.drop(
		columns=[
			'提案事項_y', '提案主体_y', '所管省庁_y', '備考', '備考_x', '備考_y', 'Unnamed: 7',
			'ワーキング・グループにおける処理方針', 'ワーキング・グループにおける処理方針_x', 'ワーキング・グループにおける処理方針_y'],
		inplace = True, errors='ignore')
	def a(x):
		try:
			tmp = x['所管省庁への検討要請日_y']
			if tmp is pd.NaT:
				tmp = x['所管省庁への検討要請日_x']
		except:
			print(x)
		return tmp
	df3['所管省庁への検討要請日_x'] = df3.apply(a, raw=False, axis=1)

	def b(x):
		try:
			tmp = x['回答取りまとめ日_x']
			if tmp == pd.NaT:
				tmp = x['回答取りまとめ日_y']
		except:
			print(x)
		return tmp
	df3['回答取りまとめ日_x'] = df3.apply(b, raw=False, axis=1)
	df3= df3.drop(columns=['所管省庁への検討要請日_y', '回答取りまとめ日_y'])

	df3.rename(columns={
		'所管省庁への検討要請日_x': '所管省庁への検討要請日',
		'回答取りまとめ日_x': '回答取りまとめ日',
		'提案事項_x': '提案事項',
		'提案主体_x': '提案主体',
		'所管省庁_x': '所管省庁',
		'所管省庁の検討結果_対応の\n分類': '所管省庁の検討結果_対応の分類'
	}
	, inplace=True)

	df3.dropna(subset=['番号'], inplace=True)
	df3['番号'] = df3['番号'].astype(int)
	df3['区分'] = item['type']
	df3['通番'] = item['type'][0] + str(item['year']) + df3['番号'].map('-{:04d}'.format)
	print(df3.columns)

	df_list.append(df3.copy())

df_all = pd.concat(df_list, ignore_index=True)

./rawdata/02_teian_kisei_20230829_R5.xlsx 3
Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考',
       'ワーキング・グループにおける処理方針'],
      dtype='object')
./rawdata/18_kaitou_kisei_20230829_R5.xlsx


C:\Users\tsasa\AppData\Local\Temp\ipykernel_4696\3681273349.py:32: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  df2.columns = ["_".join(x) if isinstance(x[1], str) and x[1][0:3] != "Unn" else x[0].replace('\n','') for x in df2.columns.ravel()]


Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', '区分', '通番'],
      dtype='object')
./rawdata/04_teian_kisei_20230829_R4.xlsx 3
Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考',
       'ワーキング・グループにおける処理方針'],
      dtype='object')
./rawdata/20_kaitou_kisei_20230829_R4.xlsx


C:\Users\tsasa\AppData\Local\Temp\ipykernel_4696\3681273349.py:32: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  df2.columns = ["_".join(x) if isinstance(x[1], str) and x[1][0:3] != "Unn" else x[0].replace('\n','') for x in df2.columns.ravel()]


Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', '区分', '通番'],
      dtype='object')
./rawdata/06_teian_kisei_20230829_R3.xlsx 3
Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考',
       'ワーキング・グループにおける処理方針'],
      dtype='object')
./rawdata/22_kaitou_kisei_20230829_R3.xlsx


...

Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', '区分', '通番'],
      dtype='object')
./rawdata/14_teian_gyoukaku_20230829_R3.xlsx 1
Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考'], dtype='object')
./rawdata/30_kaitou_gyoukaku_20230829_R3.xlsx


C:\Users\tsasa\AppData\Local\Temp\ipykernel_4696\3681273349.py:32: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  df2.columns = ["_".join(x) if isinstance(x[1], str) and x[1][0:3] != "Unn" else x[0].replace('\n','') for x in df2.columns.ravel()]


Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', '区分', '通番'],
      dtype='object')
./rawdata/16_teian_gyoukaku_20230829_R2.xlsx 2
Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考'], dtype='object')
./rawdata/32_kaitou_gyoukaku_20230829_R2.xlsx


C:\Users\tsasa\AppData\Local\Temp\ipykernel_4696\3681273349.py:32: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  df2.columns = ["_".join(x) if isinstance(x[1], str) and x[1][0:3] != "Unn" else x[0].replace('\n','') for x in df2.columns.ravel()]


Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', '区分', '通番'],
      dtype='object')


In [22]:
df_all = df_all.loc[:, ~ df_all.columns.str.contains('^Unnamed')]

In [23]:
df_all

,番号,所管省庁への検討要請日,回答取りまとめ日,提案事項,提案主体,所管省庁,提案の具体的内容,提案理由,所管省庁の検討結果_制度の現状,所管省庁の検討結果_該当法令等,所管省庁の検討結果_対応の分類,所管省庁の検討結果_対応の概要,区分,通番
0,1,2023-06-21,NaT,不動産登記オンライン申請で、申請物権の入力が不動産番号と具体的所在で異なる場合の救済措置,商業登記ゲンロン,法務省,NaN,NaN,NaN,NaN,NaN,NaN,規制改革,規5-0001
1,2,2023-06-21,2023-08-26,実質的な被用者である個人事業主への労務管理,個人,厚生労働省,外食チェーンや小売業、ネット通販などの企業は超過勤務手当等の労務管理にかかるコスト負担回避を...,これらの個人事業主には共通して売上額や配達数の厳しいノルマが課されており、それの達成状況によ...,労働基準法の「労働者」とは、職業の種類を問わず、事業又は事務所に使用される者で、賃金を支払...,労働基準法第９条,その他,制度の現状欄に記載の通りです。,規制改革,規5-0002
2,3,2023-06-21,2023-07-14,不動産登記権利者の住所欄に住民票通りのスペースを登記すること,商業登記ゲンロン,法務省,不動産登記における住所の記載は、住民票にスペースが記載されている場合であっても省略される取扱...,→スペースが省略される取扱いであるが、商業登記において商号にスペースが登記されているときはそ...,登記名義人の住所を登記する際、スペースは用いておりません。,NaN,対応不可,不動産登記において、登記名義人の住所を登記するに当たっては、不要な空白が用いられることによ...,規制改革,規5-0003
3,4,2023-06-21,2023-07-14,サ高住および見守りサービスの対象に６５歳以下の障害者(および難病患者)を加える,個人,厚生労働省\n国土交通省,障害者単身であっても安心した居住体制を構築するため、サービスの利用対象を拡大する。また難病患...,サ高住は65歳以上の高齢者を利用対象としているため、それ以下の国民は障害者であっても利用する...,サービス付き高齢者向け住宅の入居の要件につきましては、国土交通省・厚生労働省関係高齢者の居...,国土交通省・厚生労働省関係高齢者の居住の安定確保に関する法律施行規則（平成二十三年厚生労働省...,現行制度で対応可能,制度の現状欄に記載の通りです。,規制改革,規5-0004
4,5,2023-06-21,2023-07-14,国民年金保険料の学生納付特例制度の更新手続きについて,個人,厚生労働省,紙で更新手続きをしているが、それをオンライン申請に変更する。\n新たに在籍校を変更したときは...,大学を変えて新たに学生証を提示しなければならないときは紙で更新する必要性は理解できる。しか...,同じ学校に在学中である場合の２回目以降の申請については、卒業予定年月まで改めての学生証の添...,\n国民年金法第90条の3\n国民年金法施行規則第77条の4,対応,制度の現状欄に記載のとおりです。,規制改革,規5-0005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4418,1030,2021-08-29,2021-11-06,政府統計のローデータをSQLにて公開する,個人,総務省,\n政府統計全般のローデータを（個人情報保護の処理は行ったうえで）SQLサーバにて公開し、広...,\n現在の政府統計は形式がバラバラであるが、もしこれを仮にCSV等の形式としてe-Stadで...,統計を作成するために用いられる調査票情報（ローデータ）については、個人又は法人その他の団体の...,なし,現行制度下で対応可能,制度の現状欄に記載のとおりです。,行政改革,行2-1030
4419,1031,2021-08-29,NaT,文書作成時、スペースを使った均等割り付けの禁止,個人,デジタル庁,NaN,NaN,NaN,NaN,NaN,NaN,行政改革,行2-1031
4420,1032,2021-08-29,NaT,国家資格の免許証のカード化,個人,デジタル庁\n総務省\n警察庁\n金融庁\n消費者庁\n法務省\n財務省\n文部科学省\n厚...,NaN,NaN,NaN,NaN,NaN,NaN,行政改革,行2-1032
4421,1033,2021-09-04,2021-11-06,土地に関して、様々な組織が管理する情報を統合して管理すべき（縦割りの解消）,個人,国土交通省\n法務省,\n土地の現状に関して主に公的機関が関する情報（登記に関する地図（公図）、国土地理院の地図、...,\n土地の権利の確認は、まず登記記録の確認から始まる。公図は不整合のものが大変多く、実際の土...,【法務省】\n各官署において，所管する制度等に応じ，必要な情報を保有しているものと承知してい...,測量法第27条第2項、道路法第28条、土地収用法第47条の2第1項、第66条第3項、下水道法...,対応不可,【法務省】\n 制度の現状欄に記載のとおり，各官署において所管する制度等に応じて当該制度に係...,行政改革,行2-1033


## タグ付処理

In [24]:
import MeCab
tagger = MeCab.Tagger("-Ochasen -u ./dict/local_dict.dic")

In [25]:
def get_word_bag(text):
    try:
        tag = tagger.parseToNode(text)
        words = set()
        while tag:
            feature = tag.feature.split(",")
            if feature[0] == "名詞" and len(tag.surface) >= 2:
                words.add(tag.surface)
            tag = tag.next
        return list(words)
    except:
        return []

In [26]:
all_text = df_all['提案事項'] + \
           df_all['提案の具体的内容'].replace(np.nan, '', regex=True) + \
           df_all['提案理由'].replace(np.nan, '', regex=True) + \
           df_all['所管省庁の検討結果_制度の現状'].replace(np.nan, '', regex=True) + \
           df_all['所管省庁の検討結果_対応の概要'].replace(np.nan, '', regex=True) + \
           df_all['所管省庁の検討結果_該当法令等'].replace(np.nan, '', regex=True)
tmp = all_text.apply(get_word_bag)

In [27]:
import json

with open('./dict/tag.json', encoding='utf-8') as json_file:
    rules_json = json.load(json_file)

def get_summary_tag(words):
    tags = []
    rules = rules_json['rules']

    for rule in rules:
        if any(item in words for item in rule['def']):
            tags.append(rule['tag'])
    return '\n'.join(map(str,tags))

In [28]:
df_all['タグ'] = tmp.apply(get_summary_tag)

## 過去ログ参照

In [29]:
df_all.loc[df_all['所管省庁の検討結果_制度の現状'].str.match('^(番号|規制改革|行政改革)', na=False), '所管省庁の検討結果_対応の分類'] = '回答済'

## データ保存

In [30]:
# 概要
df_all['UID'] = df_all.index.map('UID{:05d}'.format)
df_all.drop_duplicates(subset=['通番'], inplace=True)
df_all[['UID','番号', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '所管省庁への検討要請日','所管省庁の検討結果_対応の分類',
        '区分', '通番','タグ']].to_json('./s3static/src/assets/data{}.json'.format(JSON_SUFFIX), orient='records', force_ascii=False)
# 詳細
df_detailed = df_all[['UID','提案の具体的内容', '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等',
       '所管省庁の検討結果_対応の分類', '所管省庁の検討結果_対応の概要']]
df_detailed = df_detailed.set_index('UID')
df_detailed.dropna(subset=['提案の具体的内容'], inplace=True) 
df_detailed.to_json('./s3static/public/data/data{}.json'.format(JSON_SUFFIX), orient='index', force_ascii=False)

In [31]:
df_all.columns

Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', '区分', '通番', 'タグ', 'UID'],
      dtype='object')

In [32]:
df_all

,番号,所管省庁への検討要請日,回答取りまとめ日,提案事項,提案主体,所管省庁,提案の具体的内容,提案理由,所管省庁の検討結果_制度の現状,所管省庁の検討結果_該当法令等,所管省庁の検討結果_対応の分類,所管省庁の検討結果_対応の概要,区分,通番,タグ,UID
0,1,2023-06-21,NaT,不動産登記オンライン申請で、申請物権の入力が不動産番号と具体的所在で異なる場合の救済措置,商業登記ゲンロン,法務省,NaN,NaN,NaN,NaN,NaN,NaN,規制改革,規5-0001,デジタル,UID00000
1,2,2023-06-21,2023-08-26,実質的な被用者である個人事業主への労務管理,個人,厚生労働省,外食チェーンや小売業、ネット通販などの企業は超過勤務手当等の労務管理にかかるコスト負担回避を...,これらの個人事業主には共通して売上額や配達数の厳しいノルマが課されており、それの達成状況によ...,労働基準法の「労働者」とは、職業の種類を問わず、事業又は事務所に使用される者で、賃金を支払...,労働基準法第９条,その他,制度の現状欄に記載の通りです。,規制改革,規5-0002,年金\nこども\n医療,UID00001
2,3,2023-06-21,2023-07-14,不動産登記権利者の住所欄に住民票通りのスペースを登記すること,商業登記ゲンロン,法務省,不動産登記における住所の記載は、住民票にスペースが記載されている場合であっても省略される取扱...,→スペースが省略される取扱いであるが、商業登記において商号にスペースが登記されているときはそ...,登記名義人の住所を登記する際、スペースは用いておりません。,NaN,対応不可,不動産登記において、登記名義人の住所を登記するに当たっては、不要な空白が用いられることによ...,規制改革,規5-0003,,UID00002
3,4,2023-06-21,2023-07-14,サ高住および見守りサービスの対象に６５歳以下の障害者(および難病患者)を加える,個人,厚生労働省\n国土交通省,障害者単身であっても安心した居住体制を構築するため、サービスの利用対象を拡大する。また難病患...,サ高住は65歳以上の高齢者を利用対象としているため、それ以下の国民は障害者であっても利用する...,サービス付き高齢者向け住宅の入居の要件につきましては、国土交通省・厚生労働省関係高齢者の居...,国土交通省・厚生労働省関係高齢者の居住の安定確保に関する法律施行規則（平成二十三年厚生労働省...,現行制度で対応可能,制度の現状欄に記載の通りです。,規制改革,規5-0004,医療,UID00003
4,5,2023-06-21,2023-07-14,国民年金保険料の学生納付特例制度の更新手続きについて,個人,厚生労働省,紙で更新手続きをしているが、それをオンライン申請に変更する。\n新たに在籍校を変更したときは...,大学を変えて新たに学生証を提示しなければならないときは紙で更新する必要性は理解できる。しか...,同じ学校に在学中である場合の２回目以降の申請については、卒業予定年月まで改めての学生証の添...,\n国民年金法第90条の3\n国民年金法施行規則第77条の4,対応,制度の現状欄に記載のとおりです。,規制改革,規5-0005,デジタル\n年金\n教育,UID00004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4418,1030,2021-08-29,2021-11-06,政府統計のローデータをSQLにて公開する,個人,総務省,\n政府統計全般のローデータを（個人情報保護の処理は行ったうえで）SQLサーバにて公開し、広...,\n現在の政府統計は形式がバラバラであるが、もしこれを仮にCSV等の形式としてe-Stadで...,統計を作成するために用いられる調査票情報（ローデータ）については、個人又は法人その他の団体の...,なし,現行制度下で対応可能,制度の現状欄に記載のとおりです。,行政改革,行2-1030,デジタル,UID04418
4419,1031,2021-08-29,NaT,文書作成時、スペースを使った均等割り付けの禁止,個人,デジタル庁,NaN,NaN,NaN,NaN,NaN,NaN,行政改革,行2-1031,,UID04419
4420,1032,2021-08-29,NaT,国家資格の免許証のカード化,個人,デジタル庁\n総務省\n警察庁\n金融庁\n消費者庁\n法務省\n財務省\n文部科学省\n厚...,NaN,NaN,NaN,NaN,NaN,NaN,行政改革,行2-1032,,UID04420
4421,1033,2021-09-04,2021-11-06,土地に関して、様々な組織が管理する情報を統合して管理すべき（縦割りの解消）,個人,国土交通省\n法務省,\n土地の現状に関して主に公的機関が関する情報（登記に関する地図（公図）、国土地理院の地図、...,\n土地の権利の確認は、まず登記記録の確認から始まる。公図は不整合のものが大変多く、実際の土...,【法務省】\n各官署において，所管する制度等に応じ，必要な情報を保有しているものと承知してい...,測量法第27条第2項、道路法第28条、土地収用法第47条の2第1項、第66条第3項、下水道法...,対応不可,【法務省】\n 制度の現状欄に記載のとおり，各官署において所管する制度等に応じて当該制度に係...,行政改革,行2-1033,デジタル,UID04421
